# Packages Installs and environment preparation

In [21]:
# !7z x data/20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS.7z
# !7z x data/20200426_UOWM_IEC104_Dataset_c_ci_na_1.7z
# !7z x data/20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS.7z
# !7z x data/20200427_UOWM_IEC104_Dataset_c_se_na_1.7z
# !7z x data/20200428_UOWM_IEC104_Dataset_c_sc_na_1.7z
# !7z x data/20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS.7z
# !7z x data/20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS.7z
# !7z x data/20200605_UOWM_IEC104_Dataset_c_rd_na_1.7z
# !7z x data/20200608_UOWM_IEC104_Dataset_mitm_drop.7z
# !7z x data/20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS.7z
# !7z x data/20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS.7z
# !7z x data/20200606_UOWM_IEC104_Dataset_c_rp_na_1.7z

# Imports

In [22]:
from scapy.all import *
import pandas as pd
# from catboost import CatBoostClassifier
# import shap
import seaborn as sns
# import feature_engine
# import feature_engine.encoding
import numpy as np
# from catboost import Pool
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os
# from google.colab import drive
# drive.mount('/content/drive')

## Code

In [23]:
def get_attack_directory_attack(path):
  return path.split('Dataset_')[1]
attacks_directories= [
                      '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1',
                      '20200427_UOWM_IEC104_Dataset_c_se_na_1',
                      '20200428_UOWM_IEC104_Dataset_c_sc_na_1',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS',
                      '20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS',
                      '20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS',
                                            # '20200608_UOWM_IEC104_Dataset_mitm_drop'
                                            ]
devices_to_read = ['qtester','attacker1','attacker2','attacker3']

# if '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
#   not
get_attack_directory_attack(attacks_directories[0])

'm_sp_na_1_DoS'

In [24]:
a={}
for attack_directory in attacks_directories:
  for device_to_read in devices_to_read:
    pcap_file = f"data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    packets=rdpcap(pcap_file,count=200)
    for packet in packets:
      a[len(packet[TCP].payload)] = 1
MAX_IEC104_PAYLOAD_BYTES = max(a)

In [25]:
MAX_IEC104_PAYLOAD_BYTES

88

In [26]:
from pathlib import Path
all_packets = []
for attack_directory in attacks_directories:
#   dfs=[]
  print(attack_directory)
  for device_to_read in devices_to_read:
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    pcap_file = f"./data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    # print(device_to_read,attack_directory)
    # df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)
    packets=rdpcap(pcap_file)
    if device_to_read != 'qtester':
      label = get_attack_directory_attack(attack_directory)
    else:
      label = 'normal'
    if device_to_read == 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      label = get_attack_directory_attack(attack_directory)
      
    all_packets.append([packets,label,get_attack_directory_attack(attack_directory)])
    # df['label'] = label
    # print(df.columns)
    # print(df.index)
    # df['flow'] =  df['flow']+'_'+df['label']
    # df.index = df.index.map(lambda x: x +'_'+ get_attack_directory_attack(attack_directory))
    # df['flow']=df['flow']
    # print(df.head())
    # dfs.append(df)
#   df = pd.concat(dfs)
#   df.to_parquet(f"./data/iec104v2_15.parquet")

20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS
20200426_UOWM_IEC104_Dataset_c_ci_na_1
20200427_UOWM_IEC104_Dataset_c_se_na_1
20200428_UOWM_IEC104_Dataset_c_sc_na_1
20200605_UOWM_IEC104_Dataset_c_rd_na_1
20200606_UOWM_IEC104_Dataset_c_rp_na_1
20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS
20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS
20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS
20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS
20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS


In [27]:
def get_sessions_with_timeout(timeout,experiment_attack_directory):
    last_packet_time = defaultdict(lambda: 0)  # Initialize last_packet_time dictionary with default value 0
    flow_bin = defaultdict(lambda: 0)
    def session_extractor(packet):
        sess = [experiment_attack_directory,"TCP", packet["IP"].src, packet["TCP"].sport,
                                   packet["IP"].dst, packet["TCP"].dport]

        sess_key = str(sorted(sess, key=str))
        current_time = packet.time

        # Track original session key time
        if current_time - last_packet_time[sess_key] > timeout:
            flow_bin[sess_key]+=1
            # If the timeout is exceeded, modify the session key to create a new session
        sess.append(flow_bin[sess_key])
        last_packet_time[sess_key] = current_time
        sess_key = str(sorted(sess, key=str))
        return sess_key
    return session_extractor

In [30]:
import binascii
def build_dataframe(packet_list):
        ip_fields = ['ip_'+field.name for field in IP().fields_desc]
        tcp_fields = ['tcp_'+field.name for field in TCP().fields_desc]
        ip_fields_raw = [field.name for field in IP().fields_desc]
        tcp_fields_raw = [field.name for field in TCP().fields_desc]
        # tcp_fields.remove('flags')
        # tcp_fields.append('tcp_flags')
        # udp_fields = [field.name for field in UDP().fields_desc]

        dataframe_fields = ip_fields + ['time'] + tcp_fields + ['size','payload','payload_raw','payload_hex']
        # dataframe_fields = list(set(dataframe_fields))

        # Create blank DataFrame
        df = pd.DataFrame(columns=dataframe_fields)
        for packet in packet_list[IP]:
            # Field array for each row of DataFrame
            field_values = []
            # Add all IP fields to dataframe
            for field in ip_fields_raw:
                if field == 'options':
                    # Retrieving number of options defined in IP Header
                    field_values.append(len(packet[IP].fields[field]))
                else:
                    field_values.append(packet[IP].fields[field])

            field_values.append(packet.time)

            layer_type = type(packet[IP].payload)
            for field in tcp_fields_raw:
                try:
                    if field == 'options':
                        field_values.append(len(packet[layer_type].fields[field]))
                    else:
                        field_values.append(packet[layer_type].fields[field])
                except:
                    field_values.append(None)
            
            # Append payload
            field_values.append(len(packet))
            field_values.append(len(packet[layer_type].payload))
            field_values.append(packet[layer_type].payload.original)
            field_values.append(binascii.hexlify(packet[layer_type].payload.original))
            # Add row to DF
            df_append = pd.DataFrame([field_values], columns=dataframe_fields)
            df = pd.concat([df, df_append], axis=0)
            
        # Reset Index
        df = df.reset_index()
        # Drop old index column
        df = df.drop(columns="index")
        return df


In [54]:
from tqdm.notebook import tqdm
import pandas as pd

TIMEOUT = 15
dfs = []

# Outer tqdm loop for processing packets
for packets, label, experiment_attack_directory in tqdm(all_packets, desc="Processing Packets"):
    # Removed print statements for cleaner output

    session_extractor = get_sessions_with_timeout(TIMEOUT, experiment_attack_directory)
    session_dict = packets.sessions(session_extractor)
    # Removed print statements for cleaner output

    session_dfs = dict()
    
    # Inner tqdm loop for processing sessions
    inner_tqdm = tqdm(session_dict.items(), desc="Processing Sessions", leave=False)
    for flow_key, flow in inner_tqdm:
        # Update the description with the current flow size
        inner_tqdm.set_description(f"Processing flow size: {len(flow)}")
        session_dfs[flow_key] = build_dataframe(flow)
        # break
        
    flow_df = pd.concat(session_dfs)
    flow_df['label'] = label
    dfs.append(flow_df)


Processing Packets:   0%|          | 0/41 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/17 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/3039 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/717 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/150 [00:00<?, ?it/s]

...

Processing Sessions:   0%|          | 0/855 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/2433 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/534 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/526 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/834 [00:00<?, ?it/s]

In [55]:
df = pd.concat(dfs)

In [56]:
df.iloc[0]

ip_version                             4
ip_ihl                                 5
ip_tos                                 0
ip_len                                46
ip_id                               2437
ip_flags                              DF
ip_frag                                0
ip_ttl                               128
ip_proto                               6
ip_chksum                              0
ip_src                      192.168.1.20
ip_dst                      192.168.1.21
ip_options                             0
time                   1587763927.678300
tcp_sport                          50045
tcp_dport                           2404
tcp_seq                       2931287365
tcp_ack                       2338122368
tcp_dataofs                            5
tcp_reserved                           0
tcp_flags                             PA
tcp_window                         16425
tcp_chksum                         33690
tcp_urgptr                             0
tcp_options     

In [57]:
df.columns

Index(['ip_version', 'ip_ihl', 'ip_tos', 'ip_len', 'ip_id', 'ip_flags',
       'ip_frag', 'ip_ttl', 'ip_proto', 'ip_chksum', 'ip_src', 'ip_dst',
       'ip_options', 'time', 'tcp_sport', 'tcp_dport', 'tcp_seq', 'tcp_ack',
       'tcp_dataofs', 'tcp_reserved', 'tcp_flags', 'tcp_window', 'tcp_chksum',
       'tcp_urgptr', 'tcp_options', 'size', 'payload', 'payload_raw',
       'payload_hex', 'label'],
      dtype='object')

In [66]:
df['ip_flags']

Flow                                                                      PacketNumber
[1, '192.168.1.20', '192.168.1.21', 2404, 50045, 'TCP', 'm_sp_na_1_DoS']  0               DF
                                                                          1               DF
                                                                          2               DF
                                                                          3               DF
                                                                          4               DF
                                                                                          ..
[1, '192.168.1.22', '192.168.1.29', 2404, 39973, 'TCP', 'c_sc_na_1_DoS']  4               DF
                                                                          5               DF
                                                                          6               DF
                                                                          7 

In [59]:
df.index[0]

("[1, '192.168.1.20', '192.168.1.21', 2404, 50045, 'TCP', 'm_sp_na_1_DoS']", 0)

In [60]:
df.index.names = ['Flow', 'PacketNumber']

In [62]:
df.groupby(level='Flow')['ip_id'].count().mean(),df.groupby(level='Flow')['ip_id'].count().median()

(10.89370599978985, 7.0)

In [68]:
df['ip_flags']=df['ip_flags'].map(lambda x: str(x))
df['tcp_flags']=df['tcp_flags'].map(lambda x: str(x))

In [69]:
df.to_parquet(f"data/iec104v2_{TIMEOUT}.parquet")

In [70]:
import scipy.stats
scipy.stats.mstats.mquantiles(df.groupby(level='Flow')['ip_id'].count(),prob=[0.25,0.5,0.75,0.9,0.99])

array([ 3.,  7., 10., 11., 21.])

In [ ]:
# df.to_parquet("/content/drive/My Drive/IEC104/iec104.parquet")